In [53]:
import pandas as pd
import earthaccess
import geopandas as gpd
import xarray as xr
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, box
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.geometry import shape
import time
import folium
import geopandas as gpd
import math
from shapely.geometry import Polygon, Point, LineString
import h5py

floripa_bounds = (-48.7242431, -27.917059, -48.3585929, -27.3804967)
floripa_bounds_polygon = box(*floripa_bounds)
floripa_center = [-27.5935, -48.5585]
dates = [("2025-01-23", "2025-01-23"), ("2025-01-30", "2025-01-30"), ("2025-02-06", "2025-02-06")]

def truncar(numero, casas_decimais):
    """Trunca um número para um número específico de casas decimais."""
    fator = 10.0 ** casas_decimais
    return math.trunc(numero * fator) / fator

def truncar_geometria(geom, casas_decimais):
    """
    Função inteligente que verifica o tipo de geometria e aplica o método
    de truncamento correto para Polígonos, Linhas e Pontos.
    """
    if geom is None or geom.is_empty:
        return None

    # Se for um Polígono, usa a lógica de exterior/interiores
    if geom.geom_type == 'Polygon':
        coords_exteriores = [(truncar(x, casas_decimais), truncar(y, casas_decimais)) for x, y in geom.exterior.coords]
        coords_interiores = []
        for interior in geom.interiors:
            coords_interiores.append([(truncar(x, casas_decimais), truncar(y, casas_decimais)) for x, y in interior.coords])
        return Polygon(coords_exteriores, coords_interiores)

    # Se for uma Linha, usa a lógica simples de .coords
    elif geom.geom_type == 'LineString':
        coords = [(truncar(x, casas_decimais), truncar(y, casas_decimais)) for x, y in geom.coords]
        return LineString(coords)

    # Se for um Ponto, usa a lógica de .x e .y
    elif geom.geom_type == 'Point':
        return Point(truncar(geom.x, casas_decimais), truncar(geom.y, casas_decimais))
        
    # Para outros tipos de geometria (MultiPolygon, etc.), retorna o original
    else:
        print(f"Aviso: Tipo de geometria '{geom.geom_type}' não suportado para truncamento. Retornando original.")
        return geom

def get_districts_loction(df_districts):
    geolocator = Nominatim(user_agent="busca_bairros_floripa_app")
    geometrias = []

    print(f"Iniciando a busca de geometrias para {len(df_districts)} bairros. Isso pode levar um minuto...")

    for bairro in df_districts['Bairro']:
        try:
            query = f"{bairro}, Florianópolis, Santa Catarina"
            location = geolocator.geocode(query, geometry='geojson')
            
            if location and 'geojson' in location.raw:
                geom = shape(location.raw['geojson'])
                geometrias.append(geom)
            else:
                geometrias.append(None)
        except Exception as e:
            geometrias.append(None)
        time.sleep(1)

    df_districts['geometry'] = geometrias
    bairros_gdf = gpd.GeoDataFrame(df_districts, geometry='geometry', crs="EPSG:4326")
    bairros_gdf.dropna(subset=['geometry'], inplace=True)

    return bairros_gdf

def download_metric(metric, range_dates, bbox, files_qty=1):
    try:
        auth = earthaccess.login(strategy="netrc")
        if not auth.authenticated:
            print("Autenticação falhou! Verifique seu arquivo .netrc.")
            return
    except Exception as e:
        print(f"Erro na autenticação: {e}")
        return
    
    # Procurando os dados GPM IMERG Half-Hourly
    # Short Name para o produto IMERG Final Run Half-Hourly V07: GPM_3IMERGHH
    results = earthaccess.search_data(
        short_name=metric,
        bounding_box=bbox,
        temporal=range_dates, # Exemplo: buscando 1 dia de dados
        count=files_qty
    )
    if not results:
        print("pesquisa sem resultados")
        return
    
    d_files = []
    for r in results:
        d_files.append(earthaccess.download(r, local_path="data"))
    return d_files

def merge_with_smap(smap_gdf: gpd.GeoDataFrame, bairros_gdf: gpd.GeoDataFrame, max_distance=5000) -> pd.DataFrame:
    smap_grouped = smap_gdf.groupby('date')

    merged_df = bairros_gdf.groupby('date').apply(
        safe_sjoin_nearest,
        smap_grouped=smap_grouped
    ).reset_index(drop=True)
    bairros_com_smap = merged_df.sort_values(by="soil_moisture", ascending=False)
    bairros_com_smap = bairros_com_smap.drop_duplicates(subset=["Bairro"], keep="first")

    bairros_com_smap = bairros_com_smap.replace(np.nan, 0)
    bairros_com_smap.reset_index(drop=True, inplace=True)
    bairros_com_smap.drop("index_right", axis=1, inplace=True)
    return bairros_com_smap

def merge_with_precipitation(df_gpm: gpd.GeoDataFrame, df_districts: gpd.GeoDataFrame, max_distance=5000):
    bairros_com_chuva = gpd.sjoin_nearest(df_districts, df_gpm, how="left", max_distance=max_distance)
    bairros_com_chuva = bairros_com_chuva.sort_values(by="precipitation_mmhr", ascending=False)
    bairros_com_chuva = bairros_com_chuva.drop_duplicates(subset=["Bairro"], keep="first")

    bairros_com_chuva = bairros_com_chuva.replace(np.nan, 0)
    bairros_com_chuva.reset_index(drop=True, inplace=True)
    bairros_com_chuva.drop("index_right", axis=1, inplace=True)
    return bairros_com_chuva

def generate_html_graph(data, center, map_name):
    m = folium.Map(location=center, zoom_start=11)

    folium.GeoJson(
        data,
        name=map_name,
        tooltip=folium.GeoJsonTooltip(fields=['Bairro'], aliases=['Bairro:']),
        style_function=lambda x: {
            'fillColor': '#4CAF50', # Cor de preenchimento verde
            'color': 'black',      # Cor da borda preta
            'weight': 1,           # Espessura da borda
            'fillOpacity': 0.4     # Opacidade do preenchimento
        }
    ).add_to(m)

    folium.LayerControl().add_to(m)

    mapa_path = f'{map_name}.html'
    m.save(mapa_path)

def get_gpm(ds):
    precip_data = ds['precipitation'].values[0]
    lats = ds['lat'].values
    lons = ds['lon'].values
    
    lon_grid, lat_grid = np.meshgrid(lons, lats)
    points = [Point(lon, lat) for lon, lat in zip(lon_grid.ravel(), lat_grid.ravel())]
    precip_values = precip_data.ravel()

    gpm_gdf = gpd.GeoDataFrame({
        'precipitation_mmhr': precip_values,
        'geometry': points
    }, crs="EPSG:4326")
    
    return gpm_gdf


def safe_sjoin_nearest(district_group, smap_grouped):
    try:
        smap_group = smap_grouped.get_group(district_group.name)

        smap_gdf = gpd.GeoDataFrame(smap_group, geometry='geometry', crs="EPSG:4326")

        return gpd.sjoin_nearest(district_group, smap_gdf, how="left")

    except KeyError:
        # Se a chave (data) não for encontrada, este bloco é executado
        result = district_group.copy()

        # Adiciona as colunas que viriam do SMAP e preenche com 0
        # sjoin_nearest também adiciona 'index_right', então incluímos ela.
        cols_from_smap = ['index_right', 'soil_moisture']
        for col in cols_from_smap:
            result[col] = 0

        return result

In [38]:
df_floripa_districts = pd.read_csv(r"data\floripa_districts.csv")
df_floripa_districts = get_districts_loction(df_floripa_districts)

Iniciando a busca de geometrias para 49 bairros. Isso pode levar um minuto...


In [5]:
files_qty = 10
dfs_gpm = []
df_gpm_final = pd.DataFrame()
df_smap_final = pd.DataFrame()
for date in dates:
    gpm_f = download_metric("GPM_3IMERGHHL", date, floripa_bounds, files_qty=files_qty)
    for f in gpm_f:
        f = f[0]
        print(f"Processing {f}...")
        try:
            ds = xr.open_dataset(f, group='Grid', engine="h5netcdf")
        except Exception as e:
            print(f"Erro ao abrir o arquivo {f}: {e}")
            continue
        ds = get_gpm(ds)
        name = f.replace(".HDF5", ".csv")
        limits = ds.geometry.within(floripa_bounds_polygon)
        ds = ds[limits]
        ds["date"] = date[0]
        ds.to_csv(name, index=False)
        dfs_gpm.append(name)

    df_gpm = pd.DataFrame()
    for gpm in dfs_gpm:
        df_temp = pd.read_csv(gpm)
        df_gpm = pd.concat([df_gpm, df_temp], ignore_index=True)
    df_gpm_final = pd.concat([df_gpm_final, df_gpm], ignore_index=True)

    smap_f = download_metric("SPL3SMP_E", date, floripa_bounds, files_qty=files_qty)
    dfs_smap = []
    for f in smap_f:
        f = f[0]
        print(f"Processing {f}...")
        with h5py.File(f, 'r') as hf:
            data_group = hf['Soil_Moisture_Retrieval_Data_AM']
            
            moisture_dataset = data_group['soil_moisture']
            lats_dataset = data_group['latitude']
            lons_dataset = data_group['longitude']
            
            moisture_data = moisture_dataset[:]
            lats = lats_dataset[:]
            lons = lons_dataset[:]
            
            fill_value = moisture_dataset.attrs.get('_FillValue', 0)

        moisture_values_flat = moisture_data.ravel()
        
        lons_flat = lons.ravel()
        lats_flat = lats.ravel()

        valid_mask = moisture_values_flat != fill_value
        points = [Point(lon, lat) for lon, lat in zip(lons_flat[valid_mask], lats_flat[valid_mask])]
        valid_moisture_values = moisture_values_flat[valid_mask]

        smap_gdf = gpd.GeoDataFrame({
            'soil_moisture': valid_moisture_values,
            'geometry': points
        }, crs="EPSG:4326")
        
        name = f.replace(".h5", ".csv")
        limits = smap_gdf.geometry.within(floripa_bounds_polygon)
        smap_gdf = smap_gdf[limits]
        smap_gdf["date"] = date[0]
        smap_gdf.to_csv(name, index=False)
        dfs_smap.append(name)

    df_smap = pd.DataFrame()
    for smap in dfs_smap:
        df_temp = pd.read_csv(smap)
        df_smap = pd.concat([df_smap, df_temp], ignore_index=True)
    df_smap_final = pd.concat([df_smap_final, df_smap], ignore_index=True)
    

df_gpm_final.to_csv(f"data/gpm.csv", index=False)
df_smap_final.to_csv(f"data/smap.csv", index=False)

smap = gpd.GeoDataFrame(
        df_smap_final,
        geometry=gpd.GeoSeries.from_wkt(df_smap_final['geometry']),
        crs="EPSG:4326"
    )
gpm = gpd.GeoDataFrame(
        df_gpm_final,
        geometry=gpd.GeoSeries.from_wkt(df_gpm['geometry']),
        crs="EPSG:4326"
    )

QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 999.83it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<00:00, 1000.31it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.48s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 500.22it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.46s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 1000.55it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.76s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 1001.51it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.46s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████|

Processing data\3B-HHR-L.MS.MRG.3IMERG.20250123-S000000-E002959.0000.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250123-S003000-E005959.0030.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250123-S010000-E012959.0060.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250123-S013000-E015959.0090.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250123-S020000-E022959.0120.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250123-S023000-E025959.0150.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250123-S030000-E032959.0180.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250123-S033000-E035959.0210.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250123-S040000-E042959.0240.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250123-S043000-E045959.0270.V07B.HDF5...


QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 999.60it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:24<00:00, 24.66s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]


Processing data\SMAP_L3_SM_P_E_20250123_R19240_002.h5...
Processing data\SMAP_L3_SM_P_E_20250123_R19240_002.h5...


QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 1607.01it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.88s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.77s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 999.83it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.56s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.38s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 999.36it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.36s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<

...

C:\Users\Bee\AppData\Local\Temp\ipykernel_7964\2607836854.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_smap_final = pd.concat([df_smap_final, df_smap], ignore_index=True)
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 1781.02it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.75s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 998.64it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.30s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 999.60it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.57s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [0

Processing data\3B-HHR-L.MS.MRG.3IMERG.20250206-S000000-E002959.0000.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250206-S003000-E005959.0030.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250206-S010000-E012959.0060.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250206-S013000-E015959.0090.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250206-S020000-E022959.0120.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250206-S023000-E025959.0150.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250206-S030000-E032959.0180.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250206-S033000-E035959.0210.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250206-S040000-E042959.0240.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250206-S043000-E045959.0270.V07B.HDF5...


QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 1004.14it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:48<00:00, 48.79s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 983.65it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 255.30it/s]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<00:00, 993.91it/s]


Processing data\SMAP_L3_SM_P_E_20250206_R19240_001.h5...
Processing data\SMAP_L3_SM_P_E_20250206_R19240_001.h5...


C:\Users\Bee\AppData\Local\Temp\ipykernel_7964\2607836854.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_smap_final = pd.concat([df_smap_final, df_smap], ignore_index=True)


In [68]:
df_floripa_districts = merge_with_precipitation(gpm, df_floripa_districts)
df_floripa_districts = merge_with_smap(smap, df_floripa_districts)

df_srtm = pd.read_csv("data/srtm.csv")
df_srtm = df_srtm[["Bairro", "Inclinacao_Max"]].rename(columns={"Inclinacao_Max": "srtm_max"})

df_floripa_districts = df_floripa_districts.merge(df_srtm, on="Bairro", how="left")
df_floripa_districts["srtm_max"] = df_floripa_districts["srtm_max"].fillna(0)

d:\Users\Bee\Estudos\nasa-challenge\nasa-challenge\lib\site-packages\geopandas\array.py:408: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
C:\Users\Bee\AppData\Local\Temp\ipykernel_7964\1877579852.py:115: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_df = bairros_gdf.groupby('date').apply(


In [69]:
def gpm_score(s):
    if s['precipitation_mmhr'] <= 20:
        return 'Baixa'
    elif s['precipitation_mmhr'] <= 50:
        return 'Moderada'
    elif s['precipitation_mmhr'] <= 100:
        return 'Alta'
    else:
        return 'Crítica'
    
def smap_score(s):
    if s['soil_moisture'] <= 0.15:
        return 'Baixa'
    elif s['soil_moisture'] <= 0.30:
        return 'Moderada'
    elif s['soil_moisture'] <= .40:
        return 'Alta'
    else:
        return 'Crítica'
    
def srtm_score(s):
    if s['srtm_max'] <= 5:
        return 'Baixa'
    elif s['srtm_max'] <= 15:
        return 'Moderada'
    elif s['srtm_max'] <= 30:
        return 'Alta'
    else:
        return 'Crítica'
    
df_floripa_districts["gpm_score"] = df_floripa_districts.apply(gpm_score, axis=1)
df_floripa_districts["smap_score"] = df_floripa_districts.apply(smap_score, axis=1)
df_floripa_districts["srtm_score"] = df_floripa_districts.apply(srtm_score, axis=1)

In [70]:
from itertools import product

niveis = ['Baixa', 'Média', 'Alta', 'Crítica']

combinacoes = list(product(niveis, repeat=3))

df_matrix = pd.DataFrame(combinacoes, columns=['srtm_score', 'gpm_score', 'smap_score'])

score_map = {'Baixa': 1, 'Média': 2, 'Alta': 3, 'Crítica': 4}

def definir_risco_e_acao(row):
    """Calcula o risco e a ação com base na soma dos scores."""
    total_score = score_map[row['srtm_score']] + \
                  score_map[row['gpm_score']] + \
                  score_map[row['smap_score']]
    
    if total_score <= 5:
        return 'Baixo', 'Observar'
    elif total_score <= 7:
        return 'Moderado', 'Monitorar'
    elif total_score <= 9:
        return 'Alto', 'Alertar'
    else: # total_score >= 10
        return 'Crítico', 'Evacuar'

df_matrix[['risk_classification', 'recommended_action']] = df_matrix.apply(
    definir_risco_e_acao, axis=1, result_type='expand'
)

df_floripa_districts = pd.merge(
    df_floripa_districts,      # O DataFrame da esquerda (principal)
    df_matrix,                 # O DataFrame da direita (tabela de consulta)
    on=['srtm_score', 'gpm_score', 'smap_score'],  # As colunas-chave para a correspondência
    how='left'                 # Tipo de junção: mantém tudo da esquerda
)

df_floripa_districts.loc[df_floripa_districts.risk_classification.isna(), "risk_classification"] = "Moderado"
df_floripa_districts.loc[df_floripa_districts.risk_classification.isna(), "recommended_action"] = "Monitorar"

df_floripa_districts["lat"] = df_floripa_districts.geometry.centroid.x
df_floripa_districts["lon"] = df_floripa_districts.geometry.centroid.y
df_floripa_districts.to_csv("data/data.csv", index=False)

C:\Users\Bee\AppData\Local\Temp\ipykernel_7964\3045360064.py:40: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_floripa_districts["lat"] = df_floripa_districts.geometry.centroid.x
C:\Users\Bee\AppData\Local\Temp\ipykernel_7964\3045360064.py:41: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_floripa_districts["lon"] = df_floripa_districts.geometry.centroid.y


In [72]:
df_floripa_districts["risk_classification"].value_counts()

risk_classification
Moderado    37
Baixo       12
Name: count, dtype: int64

In [63]:
df_floripa_districts.columns

Index(['Bairro', 'geometry', 'precipitation_mmhr', 'date', 'soil_moisture',
       'srtm_max', 'gpm_score', 'smap_score', 'srtm_score',
       'risk_classification_x', 'recommended_action_x', 'Média', 'lat', 'lon',
       'risk_classification_y', 'recommended_action_y'],
      dtype='object')

In [64]:
df_floripa_districts

,Bairro,geometry,precipitation_mmhr,date,soil_moisture,srtm_max,gpm_score,smap_score,srtm_score,risk_classification_x,recommended_action_x,Média,lat,lon,risk_classification_y,recommended_action_y
0,Campeche,"POLYGON ((-48.5245 -27.67651, -48.5245 -27.676...",11.309999,2025-01-30,0,35.74,Baixa,Baixa,Crítica,Moderado,Monitorar,NaN,-48.497349,-27.667491,Moderado,Monitorar
1,Caieira da Barra do Sul,"POLYGON ((-48.56401 -27.81713, -48.56397 -27.8...",10.179999,2025-01-30,0,37.15,Baixa,Baixa,Crítica,Moderado,Monitorar,NaN,-48.563893,-27.816965,Moderado,Monitorar
2,Morro das Pedras,"POLYGON ((-48.51527 -27.71372, -48.51201 -27.7...",9.599999,2025-01-30,0,33.40,Baixa,Baixa,Crítica,Moderado,Monitorar,NaN,-48.506472,-27.712434,Moderado,Monitorar
3,Tapera,POINT (-48.56128 -27.78806),9.599999,2025-01-30,0,37.84,Baixa,Baixa,Crítica,Moderado,Monitorar,NaN,-48.561275,-27.788064,Moderado,Monitorar
4,Pântano do Sul,"POLYGON ((-48.55926 -27.76745, -48.55876 -27.7...",9.599999,2025-01-30,0,33.57,Baixa,Baixa,Crítica,Moderado,Monitorar,NaN,-48.523214,-27.762588,Moderado,Monitorar
5,Armação do Pântano do Sul,"POLYGON ((-48.53388 -27.74716, -48.53331 -27.7...",9.599999,2025-01-30,0,40.71,Baixa,Baixa,Crítica,Moderado,Monitorar,NaN,-48.514003,-27.744163,Moderado,Monitorar
6,Vargem Grande,"POLYGON ((-48.45818 -27.46639, -48.4577 -27.46...",9.500000,2025-01-30,0,32.10,Baixa,Baixa,Crítica,Moderado,Monitorar,NaN,-48.442200,-27.471467,Moderado,Monitorar
7,Canasvieiras,"POLYGON ((-48.54722 -27.45903, -48.54715 -27.4...",9.500000,2025-01-30,0,31.46,Baixa,Baixa,Crítica,Moderado,Monitorar,NaN,-48.486608,-27.449536,Moderado,Monitorar
8,Vargem Pequena,"POLYGON ((-48.48808 -27.48004, -48.4875 -27.48...",9.500000,2025-01-30,0,30.51,Baixa,Baixa,Crítica,Moderado,Monitorar,NaN,-48.459723,-27.490630,Moderado,Monitorar
9,Vargem do Bom Jesus,"POLYGON ((-48.449 -27.44439, -48.44472 -27.449...",9.500000,2025-01-30,0,39.26,Baixa,Baixa,Crítica,Moderado,Monitorar,NaN,-48.430571,-27.450635,Moderado,Monitorar
